In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import os
import re
sys.path.append("../scripts")
from joinfiles import JoinFiles

# Correccion de datos

en base a los errores identificados, realizare una limpieza de datos buscando conservar la mayor cantidad de archivos.

In [2]:
# cargamos los nombres de los archivos con problemas y los errores asociados
data_path = "../data"
files = np.load(data_path + "/bad_files.npy")
errors = np.load(data_path + "/errors.npy")

### Correccion de headers

corregido manualmente, son archivos excel donde por interaccion humana se modifico su estructura

In [3]:

error_type = "ERROR.badHeader"
idxs = np.where(errors == error_type)[0]
for idx in idxs:
    print(files[idx])
    
print("luego de la correccion...")

for idx in idxs:
    df = pd.read_excel(files[idx])
    print(df.columns)
    print("--------")

luego de la correccion...


### Correcion de formato

varios archivos que no pudieron ser leidos es porque fueron mal pasados, pero algunos presentan solo un error de formato, siendo .csv pero con extension .xls
Los archivos malos son borrados y los con error de formato son corregidos (pasados a excel)

In [4]:
error_type = "ERROR.file"
idxs = np.where(errors == error_type)[0]
for idx in idxs:
    print(files[idx])
    
print("\n\nluego de la correccion...\n")

ffs = []
for idx in idxs:
    f = files[idx]
    f = f[:len(f) - 4]
    f += ".csv"
    if os.path.exists(f):
        print(f + ": FORMATEADO A .csv")
        ffs.append(f)
    else:
        print(f + ": BORRADO")



luego de la correccion...



### Correccion de archivo vacio

estos directamente son borrados

In [5]:
error_type = "ERROR.NoResultFile"

idxs = np.where(errors == error_type)[0]
for idx in idxs:
    print(files[idx])

print("\n\nluego de la correccion...\n")

for idx in idxs:
    f = files[idx]
    if not os.path.exists(f):
        print(f + ": BORRADO")
    else:
        print(f + ": FALTA POR SER BORRADO")



luego de la correccion...



### Correccion de archivo con NaN

se borran las filas con problemas

In [6]:
error_type = "ERROR.NaN"

idxs = np.where(errors == error_type)[0]
for idx in idxs:
    print(files[idx])
    
# borramos filas y guardamos el mismo archivo
print("\n\n")
for idx in idxs:
    f = files[idx]
    print("corrigiendo file %s ..." % f, end="")
    df = pd.read_excel(f)
    df = df.dropna()
    df.to_excel(f)
    print("done")

### Correccion de archivos con datos malos

es necesario verificar la extension de dicho error. existen 3 casos:
- la estacion presenta puros ceros porque no mide dichos valores
- en un intervalo pequeño de un dia fallo la estacion, se descarta el intervalo
- en un intervalo grande de un dia fallo la estacion, se descarta dia completo


#### Arbitrariamente se elige:
-  una estacion (un archivo que contiene 6 meses )con mas del 5% de mediciones mala signifca una mala estacion.
- si la estacion tiene menos del 5% de datos malos, se limpian dichos datos +- 1 hora de medicion por posibles errores escondidos (+- 4 datos)

In [7]:
error_prefix = "INCONSISTENCY"

idxs = []
for i, err in enumerate(errors):
    if error_prefix in err:
        idxs.append(i)
for idx in idxs:
    print(files[idx], errors[idx])

../data/PAULA/COQUIMBO/CARACHILLA/Carachilla01-01-2009-30-06-2009.csv INCONSISTENCY.ZeroOrNegPAtm


In [9]:
for idx in idxs:
    f = files[idx]
    err = errors[idx]
    if os.path.exists(f):
        if ".csv" in f:
            df = pd.read_csv(f, sep='\t', lineterminator='\r', decimal=",")
        else:
            df = pd.read_excel(f)
        before = len(df)
        if "NegVel" in err:
            df2 = df[df["Velocidad de Viento"] < 0]
        elif "NegDir" in err:
            df2 = df[df["Direccion de Viento"] < 0]
        elif "NegHum" in err:
            df2 = df[df["Humedad"] < 0]
        elif "ZeroOrNegPAtm" in err:
            df2 = df[df["Presion Atmosferica"] <= 0]
        after = len(df2)
        if before * 0.05 < after:
            print(f, "se descarta archiovo")
            os.remove(f)
        elif after == 0:
            print(f, "archivo ya corregido")
        else:
            print(f, "se borran datos")
            indexs = df2.index
            idxs2 = []
            for i in indexs:
                idxs2.append(i)
                for j in [1, 2, 3, 4]:
                    left = max(0, i - j)
                    right = min(before - 1, i + j)
                    idxs2.extend([left, right])
            idxs2 = np.unique(idxs2)
            idxs2 = np.sort(idxs2)
            df3 = df.drop(df.index[idxs2])
            df3.to_excel(f)
    else:
        print(f, "archivo borrado")

../data/PAULA/COQUIMBO/CARACHILLA/Carachilla01-01-2009-30-06-2009.csv archivo borrado


### Correccion de estaciones con mas de un codigo

se borra la estacion completa

In [10]:
error_type = "ERROR.notUniqueCodeInStation"

idxs = np.where(errors == error_type)[0]
for idx in idxs:
    print(files[idx])
    

print("\n\n")    
for idx in idxs:
    f = files[idx]
    if not os.path.exists(f):
        print(f, "borrado")
    else:
        print(f, "falta por borrar")

### Correccion de nombres de archivos

pandas tiene problemas con caracteres especiales como ñ o letras con acento. Para evitar esto, identificaremos a las regiones con nombres asi y los modificaremos:
- quitando los acentos
- remplazando las ñ por nh

Esto se hara manualmente

In [16]:
# cuando ya no queden problemas, este codigo no mostrara nada


files = glob.glob("../data/PAULA/*/*", recursive=True)
stations = []
full = []
for f in files:
    f = f.replace("\\", "/")
    if all(x not in f for x in ["FDF semestre 2 2010", "DE MALLANES Y LA ANTARTIDA"]):
        stations.append(f.split("/")[-1])
        full.append(f)


for s, f in zip(stations, full):
    if not re.match('^[a-zA-Z0-9 -]+$', s):
        print(f)

# Creacion de nuevo dataset

se agrupan los archivos por estacion como se realizo con anterioridad.

en este caso tambien tenemos que algunos archivos son .csv

NOTA: caracteres especiales en el nombre del archivo producen problemas en pandas. Para evitar ello, use las estaciones como carpetas, las cuales cada una tiene 1 solo archivo csv

In [3]:
raw = data_path + "/PAULA"
cleared = data_path + "/cleared"
jf = JoinFiles(raw, cleared)
jf.process_all()

:::::: ARICA Y PARINACOTA ::::::
creating: ../data/cleared/ARICA Y PARINACOTA/AZAPA1.csv
creating: ../data/cleared/ARICA Y PARINACOTA/AZAPA2.csv
creating: ../data/cleared/ARICA Y PARINACOTA/COLONIA JFLLUTA BAJO.csv
creating: ../data/cleared/ARICA Y PARINACOTA/PUROCHILE LLUTA MEDIO.csv
creating: ../data/cleared/ARICA Y PARINACOTA/SOCOROMA.csv
creating: ../data/cleared/ARICA Y PARINACOTA/TRUFA AZAPA MEDIO.csv
:::::: ATACAMA ::::::
creating: ../data/cleared/ATACAMA/ALTO DEL CARMEN.csv
creating: ../data/cleared/ATACAMA/BODEGA.csv
creating: ../data/cleared/ATACAMA/COPIAPO.csv
creating: ../data/cleared/ATACAMA/HORNITOS.csv
creating: ../data/cleared/ATACAMA/JOTABECHE.csv
creating: ../data/cleared/ATACAMA/TRANQUE-LAUTARO.csv
creating: ../data/cleared/ATACAMA/VALLENAR.csv
:::::: AYSEN DEL GENERAL DEL C.I.CAMPO ::::::
creating: ../data/cleared/AYSEN DEL GENERAL DEL C.I.CAMPO/AYSEN.csv
creating: ../data/cleared/AYSEN DEL GENERAL DEL C.I.CAMPO/CHILECHICO.csv
creating: ../data/cleared/AYSEN DEL GEN

creating: ../data/cleared/METROPOLITANA/MALLARAUCO.csv
creating: ../data/cleared/METROPOLITANA/MELIPILLA.csv
creating: ../data/cleared/METROPOLITANA/PIRQUE.csv
creating: ../data/cleared/METROPOLITANA/SAN PEDRO.csv
creating: ../data/cleared/METROPOLITANA/TALAGANTE.csv
creating: ../data/cleared/METROPOLITANA/TILTIL.csv
:::::: VALPARAISO ::::::
creating: ../data/cleared/VALPARAISO/CABILDO.csv
creating: ../data/cleared/VALPARAISO/CALLELARGA.csv
creating: ../data/cleared/VALPARAISO/CASA BLANCA.csv
creating: ../data/cleared/VALPARAISO/CATEMU.csv
creating: ../data/cleared/VALPARAISO/HIJUELAS.csv
creating: ../data/cleared/VALPARAISO/LA CRUZ.csv
creating: ../data/cleared/VALPARAISO/LIMACHE.csv
creating: ../data/cleared/VALPARAISO/LLAY-LLAY.csv
creating: ../data/cleared/VALPARAISO/LOS ANDES.csv
creating: ../data/cleared/VALPARAISO/NOGALES.csv
creating: ../data/cleared/VALPARAISO/OLMUE.csv
creating: ../data/cleared/VALPARAISO/PANQUEHUE.csv
creating: ../data/cleared/VALPARAISO/PETORCA.csv
creating

### Verificacion de nuevo dataset

verificamos que se hayan corregido todos los errores



In [4]:
from badfiles import BadFiles

In [5]:
raw_data_path = "../data/cleared"

bf = BadFiles(raw_data_path)
bf.recheck_all()
# no muestra nada si esta todo ok